In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
from tensorflow.keras import backend as keras_backend
import numpy as np

gpu_on = False

if gpu_on :
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

print(gpu_devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
from src.data_loader.classification import ClassifyDataloader
from tensorflow.keras.applications.inception_v3 import preprocess_input
from glob import glob

task = "classification"
data_set_name = "detect_lvi"
batch_size = 16
on_memory = False
# target_size = (512,512)
target_size = None
interpolation = "bilinear"
class_mode = "binary"
# class_mode = "categorical"
dtype="float32"


train_image_path_list = glob(f"./datasets/{task}/{data_set_name}/train/*/*")
valid_image_path_list = glob(f"./datasets/{task}/{data_set_name}/valid/*/*")
test_image_path_list = glob(f"./datasets/{task}/{data_set_name}/test/*/*")
label_list = os.listdir(f"./datasets/{task}/{data_set_name}/train")

label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}

train_data_loader = ClassifyDataloader(image_path_list=train_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=on_memory,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,
                                       shuffle=True,
                                       class_mode=class_mode,
                                       dtype=dtype
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=False,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)
test_data_loader = ClassifyDataloader(image_path_list=test_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=False,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                      
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)

print(f"train_num: {len(train_image_path_list)}")
print(f"valid_num: {len(valid_image_path_list)}")
print(f"test_num: {len(test_image_path_list)}")

Total data num 8608 with 2 classes
Total data num 1609 with 2 classes
Total data num 1469 with 2 classes
train_num: 8608
valid_num: 1609
test_num: 1469


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

grad_cam = False
transfer_learning = False
epoch_release_frozen = 10
transfer_train_mode = "include_deep_layer"
layer_name_frozen_to = "mixed4"

#  binary_sigmoid, categorical_sigmoid, categorical_softmax
activation = "binary_sigmoid"

# create the base pre-trained model~
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(None,None,3),
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if train_mode == "dense_only":
        base_model.trainable = False
    elif train_mode == "include_deep_layer":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == layer_name_frozen_to:
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)

if grad_cam:
    x *= 1e-1
    keras_backend.set_floatx('float64')
    dense_dtype = "float64"
else:
    dense_dtype = "float32"
    
if activation == "binary_sigmoid":
    predictions = Dense(1, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = BinaryCrossentropy(label_smoothing=0.01)
elif activation == "categorical_sigmoid":
    predictions = Dense(2, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)
elif activation == "categorical_softmax":
    predictions = Dense(2, activation='softmax', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)

# this is the model we will train
model = Model(base_model.input, predictions)

In [ ]:
from datetime import date

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Nadam

today = date.today()

# YY/MM/dd
today_str = today.strftime("%Y-%m-%d")
today_weight_path = f"./weights/{task}/{data_set_name}/{today_str}/" 
today_logs_path = f"./logs/{task}/{data_set_name}/{today_str}/"
os.makedirs(today_weight_path, exist_ok=True)
os.makedirs(today_logs_path, exist_ok=True)
optimizer = Nadam(1e-3, clipnorm=1)

save_c = ModelCheckpoint(
    today_weight_path+"/weights_{epoch:02d}_{loss:.4f}.hdf5",
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=20,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-7)
csv_logger = CSVLogger(f'{today_logs_path}/log.csv', append=False, separator=',')


def make_model_trainable(epoch, model, target_epoch):
    if epoch > target_epoch:
        for layer in model.layers:
            layer.trainable = True
            
make_trainable_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: make_model_trainable(epoch, model, target_epoch=epoch_release_frozen)
)

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
start_epoch = 0
epochs = 200

model.fit(
    train_data_loader,
    validation_data=valid_data_loader,
    epochs=epochs,
    callbacks=[reduceLROnPlat, save_c, csv_logger, make_trainable_callback],
    initial_epoch=start_epoch
)

In [ ]:
today_weight_path

In [ ]:
target_data_loader = test_data_loader

target_data_loader.batch_size = 1
model.load_weights('./weights/classification/detect_lvi/2021/06/25/'+"weights_26_0.0724.hdf5")
model_predicted = model.predict(target_data_loader)
ground_truth = np.array([label for img_array, label in target_data_loader])

model.evaluate(target_data_loader)

In [ ]:
negative_group_1 = np.where(model_predicted < 0.5)[0]
negative_group_2 = np.where(model_predicted != 0)[0]
negative_confirm = np.where(model_predicted == 0)[0]
negative_not_confirm = np.intersect1d(negative_group_1, negative_group_2)

positive_group_1 = np.where(model_predicted > 0.5)[0]
positive_group_2 = np.where(model_predicted != 1)[0]
postive_confirm = np.where(model_predicted == 1)[0]
postive_not_confirm = np.intersect1d(positive_group_1, positive_group_2)

ground_truth_negative = np.where(ground_truth == 0)[0]
ground_truth_positive = np.where(ground_truth == 1)[0]

true_negative = np.intersect1d(negative_group_1, ground_truth_negative)
true_positive = np.intersect1d(positive_group_1, ground_truth_positive)
false_negative = np.intersect1d(negative_group_1, ground_truth_positive)
false_positive = np.intersect1d(positive_group_1, ground_truth_negative)


In [ ]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

In [ ]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

In [ ]:
test_image_path_list[-10:]

In [ ]:
print(false_negative)
print(false_positive)

In [3]:
test_data_loader.batch_size = 16

test_data_loader = ClassifyDataloader(image_path_list=test_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       on_memory=False,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                      
                                       shuffle=False,
                                       class_mode=class_mode,
                                       dtype=dtype
)

Total data num 1469 with 2 classes


In [4]:
for _ in test_data_loader:
    pass

0
0
16
16
1
16
32
16
2
32
48
16
3
48
64
16
4
64
80
16
5
80
96
16
6
96
112
16
7
112
128
16
8
128
144
16
9
144
160
16
10
160
176
16
11
176
192
16
12
192
208
16
13
208
224
16
14
224
240
16
15
240
256
16
16
256
272
16
17
272
288
16
18
288
304
16
19
304
320
16
20
320
336
16
21
336
352
16
22
352
368
16
23
368
384
16
24
384
400
16
25
400
416
16
26
416
432
16
27
432
448
16
28
448
464
16
29
464
480
16
30
480
496
16
31
496
512
16
32
512
528
16
33
528
544
16
34
544
560
16
35
560
576
16
36
576
592
16
37
592
608
16
38
608
624
16
39
624
640
16
40
640
656
16
41
656
672
16
42
672
688
16
43
688
704
16
44
704
720
16
45
720
736
16
46
736
752
16
47
752
768
16
48
768
784
16
49
784
800
16
50
800
816
16
51
816
832
16
52
832
848
16
53
848
864
16
54
864
880
16
55
880
896
16
56
896
912
16
57
912
928
16
58
928
944
16
59
944
960
16
60
960
976
16
61
976
992
16
62
992
1008
16
63
1008
1024
16
64
1024
1040
16
65
1040
1056
16
66
1056
1072
16
67
1072
1088
16
68
1088
1104
16
69
1104
1120
16
70
1120
1136
16
71
1136
1152


In [5]:
test_data_loader[91]

91
1456
1469
13


(array([[[[ 0.88235295,  0.54509807,  0.77254903],
          [ 0.81960785,  0.48235297,  0.77254903],
          [ 0.8745098 ,  0.62352943,  0.85882354],
          ...,
          [ 0.9843137 ,  0.96862745,  0.9764706 ],
          [ 1.        ,  0.9843137 ,  0.99215686],
          [ 0.9843137 ,  0.96862745,  0.9764706 ]],
 
         [[ 0.8745098 ,  0.5921569 ,  0.8352941 ],
          [ 0.78039217,  0.4666667 ,  0.8509804 ],
          [ 0.8509804 ,  0.4901961 ,  0.8666667 ],
          ...,
          [ 0.99215686,  0.9764706 ,  0.9843137 ],
          [ 0.9843137 ,  0.96862745,  0.9764706 ],
          [ 1.        ,  0.9607843 ,  0.96862745]],
 
         [[ 0.30980396,  0.09019613,  0.5764706 ],
          [ 0.3411765 ,  0.14509809,  0.6784314 ],
          [ 0.45098042,  0.18431377,  0.6784314 ],
          ...,
          [ 0.99215686,  0.9843137 ,  0.9764706 ],
          [ 0.9843137 ,  0.9843137 ,  0.9607843 ],
          [ 0.9764706 ,  0.9843137 ,  0.99215686]],
 
         ...,
 
         [[ 